In [60]:
import numpy as np
import pandas as pd

In [107]:
split_num = 3
split_pair_path = "../data/split_result/split_pair_{0}.txt".format(split_num)

split_data = pd.read_csv(split_pair_path, sep='\t', header=None)

In [108]:
split_yes_no_list = np.array(split_data.iloc[:,2])
result = sum(split_yes_no_list==0)
print(result)

640702


In [112]:
%%time
k = 20
top =1

total_single_split_num = int(len(split_yes_no_list)/20)

counter = 0
for i in range(0, len(split_yes_no_list), 20):
    single_split = split_yes_no_list[i:(i+1)*top]
    if 1 in single_split:
        counter += 1
print("recall: {0}/{1}".format(counter, total_single_split_num))

recall: 9304/36746
CPU times: user 100 ms, sys: 0 ns, total: 100 ms
Wall time: 200 ms


In [99]:
len(split_yes_no_list)

735000

split_num = 1
recall: 36746/36746
positive sample: 95259/734920  = 0.1296
negative sample: 639661/734920 = 0.8703

split_num = 2
recall:  /36746

split_num = 3
recall: 36745/36746
positive sample: 94218/734920  = 0.1282
negative sample: 640702/734920 = 0.8717

split_num = 4
recall: 36744/36746
positive sample: 94365/734920  = 0.1284
negative sample: 640555/734920 = 0.8715

split_num = 5
recall: 36744/36746
positive sample: 94157/734920  = 0.1281
negative sample: 640763/734920 = 0.8718

split_num = 6
recall: 36744/36746
positive sample: 93945/734920  = 0.1278
negative sample: 640975/734920 = 0.8721

split_num = 7
recall: 36747/36750
positive sample: 92982/735000  = 0.1265
negative sample: 642018/735000 = 0.8734